# Get descriptors for molecules 
## Option 1 : extract from the autoqchem database

In [1]:
import pandas as pd
import os
#import re

import sys
sys.path.append('../')

from aqc_utils.molecule import molecule
from aqc_utils.db_functions import *
from aqc_utils.openbabel_functions import *

import hashlib
import logging
logging.basicConfig(level=logging.INFO)

## select all the molecules in autoqchem database

In [2]:
mol_df = db_select_molecules()
# if no tags => all
#tags = ['JS_substrate', 'test_JulesSchleinitz'])

/Users/julesschleinitz/Desktop/These/Projet_stage_M1_Max/Code/SM_DFT/DFT_for_SM/notebooks/../aqc_utils/db_functions.py:167: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df = pd.concat([grouped['metadata', 'molecule_id', 'name'].first(),


# generate csv with descriptors for the C-O subtrates first

In [3]:
data_df = pd.read_csv('../data_csv/Data_test09032021.csv', sep = ',') 
unik_smi = np.unique(data_df["Reactant Smile (C-O)"].tolist())
can_smis = np.unique([Chem.CanonSmiles(smi) for smi in unik_smi])

In [4]:
# drop all the molecules that are not in can_smi
idx_todrop = []
can_db = []
for j, smi in enumerate(mol_df["can"]):
    try:
        can_smi = Chem.MolToSmiles(Chem.MolFromSmiles(smi)) 
        if can_smi not in can_smis:
            idx_todrop.append(j)
        else:
            can_db.append((smi, can_smi))
    except:
        idx_todrop.append(j)
        
CO_df = mol_df.drop(axis=0, index=idx_todrop)

# drop all the molecules that don't have the good calculation setup
dft_set = {'gaussian_config': {'theory': 'b3lyp',
                               'light_basis_set': '6-31G*',
                               'heavy_basis_set': 'LANL2DZ',
                               'generic_basis_set': 'genecp',
                               'max_light_atomic_number': 36},
           'gaussian_tasks': ['opt b3lyp/6-31G* scf=(xqc,tight)',
                              'freq b3lyp/6-31G* volume NMR pop=NPA density=current Geom=AllCheck Guess=Read',
                              'TD(NStates=10, Root=1) b3lyp/6-31G* volume pop=NPA density=current Geom=AllCheck Guess=Read'],
           'max_num_conformers': 1,
           'class': '',
           'subclass': '',
           'type': '',
           'subtype': ''}

N = len(idx_todrop)

idx_todrop = []
for j, dft in enumerate(CO_df["metadata"]):
    if dft != dft_set:
        idx_todrop.append(CO_df.iloc[[j]].index[0])

CO_df = CO_df.drop(axis=0, index=idx_todrop)

N += len(idx_todrop)
if len(mol_df) - len(idx_todrop) == len(can_smis):
    print("Sounds Good")
else:
    print("not all molecules have been calculated...  ", 
          len(can_smis)- (len(mol_df) - N),
          " are missing")

RDKit ERROR: [17:51:15] Explicit valence for atom # 16 N, 4, is greater than permitted
[17:51:15] Explicit valence for atom # 16 N, 4, is greater than permitted
RDKit ERROR: [17:51:15] Explicit valence for atom # 16 N, 4, is greater than permitted
RDKit ERROR: [17:51:15] Explicit valence for atom # 17 N, 4, is greater than permitted
RDKit ERROR: [17:51:15] Explicit valence for atom # 17 N, 4, is greater than permitted
RDKit ERROR: [17:51:15] Explicit valence for atom # 17 N, 4, is greater than permitted
RDKit ERROR: [17:51:15] Explicit valence for atom # 17 N, 4, is greater than permitted
[17:51:15] Explicit valence for atom # 16 N, 4, is greater than permitted
[17:51:15] Explicit valence for atom # 17 N, 4, is greater than permitted
[17:51:15] Explicit valence for atom # 17 N, 4, is greater than permitted
[17:51:15] Explicit valence for atom # 17 N, 4, is greater than permitted
[17:51:15] Explicit valence for atom # 17 N, 4, is greater than permitted
RDKit ERROR: [17:51:15] Explicit v

not all molecules have been calculated...   39  are missing


In [5]:
tags_coll = db_connect("tags")
mols_coll = db_connect("molecules")
log_files_coll = db_connect("log_files")

In [6]:
# helper function to create file names the same way ACQ does
def mol_fs_name(can):
    mol = input_to_OBMol(can, "string", "smi")
    return f"{mol.GetFormula()}_{hashlib.md5(can.encode()).hexdigest()[:4]}"

In [7]:
# add fs_name to the table
CO_df['file_base_name'] = CO_df['can'].map(mol_fs_name)

In [8]:
# get a cursor that iterates over the log files
cursor = log_files_coll.find({'molecule_id' :{"$in": CO_df.molecule_id.tolist()}}, {'log':1, 'can':1})

In [9]:
# path for file storage
path = '/Users/julesschleinitz/Desktop/These/Projet_stage_M1_Max/Code/SM_DFT/DFT_for_SM/log_data'

In [10]:
# dictionary to keep trach of conformer numbers
m_dict = {m:0 for m in CO_df.can.tolist()}
L = !ls {path}

all_mol_dscptrs = []
for l in cursor:
    can, log = l['can'], l['log']
    fs_name = mol_fs_name(can)  
    
    with open(f"{path}/{fs_name}_{m_dict[can]}.log", "w") as f:
        f.write(log)
        log_name = f"{path}/{fs_name}_{m_dict[can]}.log"
        extractor = gaussian_log_extractor(log_name)
        extractor.__init__
        mol_descriptors = extractor.get_descriptors()
        all_mol_dscptrs.append(mol_descriptors)
        m_dict[can] += 1
        os.remove(log_name)
CO_df["descriptors_full"] = all_mol_dscptrs

In [11]:
CO_df

,can,metadata,molecule_id,name,tag,_ids,weights,num_conformers,file_base_name,descriptors_full
2226,C(Oc1ccccc1)Oc1ccccc1,"{'gaussian_config': {'theory': 'b3lyp', 'light...",615c6daaba3ae7d01d7edd5f,None,[JS_substrate],[615c6daaba3ae7d01d7edd63],[1.0],1,C13H12O2_d005,"{'descriptors': {'number_of_atoms': 19, 'charg..."
2415,C=CC[Si](Oc1ccc2c(c1)cccc2)(C)C,"{'gaussian_config': {'theory': 'b3lyp', 'light...",615461e373112bcf7451806b,None,[JS_substrate],[615461e373112bcf7451806f],[1.0],1,C15H18OSi_a4e1,"{'descriptors': {'number_of_atoms': 22, 'charg..."
2708,CC(=O)Oc1c2ccccc2cc2c1cccc2,"{'gaussian_config': {'theory': 'b3lyp', 'light...",615c6dadba3ae7d01d7edd67,None,[JS_substrate],[615c6dadba3ae7d01d7edd6b],[1.0],1,C16H12O2_e52d,"{'descriptors': {'number_of_atoms': 23, 'charg..."
2715,CC(=O)Oc1ccc(cc1)C(=O)C,"{'gaussian_config': {'theory': 'b3lyp', 'light...",615c6dafba3ae7d01d7edd6f,None,[JS_substrate],[615c6db0ba3ae7d01d7edd73],[1.0],1,C10H10O3_f806,"{'descriptors': {'number_of_atoms': 20, 'charg..."
2722,CC(=O)Oc1ccc2c(c1)ccc(c2)C(=O)C,"{'gaussian_config': {'theory': 'b3lyp', 'light...",615461e673112bcf74518073,None,[JS_substrate],[615461e773112bcf74518077],[1.0],1,C14H12O3_d929,"{'descriptors': {'number_of_atoms': 28, 'charg..."
...,...,...,...,...,...,...,...,...,...,...
16849,OCCOc1ccc(cc1)C,"{'gaussian_config': {'theory': 'b3lyp', 'light...",615c836c51214b61589ce4d9,None,[JS_substrate],[615c836d51214b61589ce4dd],[1.0],1,C9H12O2_8950,"{'descriptors': {'number_of_atoms': 39, 'charg..."
16851,OCCOc1ccccc1,"{'gaussian_config': {'theory': 'b3lyp', 'light...",615c836f51214b61589ce4e1,None,[JS_substrate],[615c836f51214b61589ce4e5],[1.0],1,C8H10O2_653a,"{'descriptors': {'number_of_atoms': 26, 'charg..."
16857,OCCc1ccc(cc1)OC,"{'gaussian_config': {'theory': 'b3lyp', 'light...",615312ccb880f911fcc68e75,None,[test_JulesSchleinitz],[615312cdb880f911fcc68e79],[1.0],1,C9H12O2_df4c,"{'descriptors': {'number_of_atoms': 23, 'charg..."
16861,OCCc1ccccc1OC,"{'gaussian_config': {'theory': 'b3lyp', 'light...",615c837151214b61589ce4e9,None,[JS_substrate],[615c837251214b61589ce4ed],[1.0],1,C9H12O2_d4d9,"{'descriptors': {'number_of_atoms': 20, 'charg..."


In [12]:
CO_df.to_csv('../data_csv/aqc_descriptors_CO.csv')